In [1]:
import numpy as np
import cv2


from keras.models import load_model
model = load_model('my_model.h5')
#model.summary()
model.load_weights('my_model_weights.h5')

#如果 load_model 有錯誤訊息, 請檢查 keras 和 tensorflow 的版本.
#建議使用 keras 2.2.4, tensorflow 1.8.0 or 1.12.0

In [2]:
img_1 = cv2.imread(r'C:\Users\timch\MyPython\opencv_test\12345.png')
img_2 = img_1[0:480, 80:560]
imgContour = img_2.copy()
cv2.imshow('Camera', img_2)

# canny
img_2_gray = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(img_2_gray, 150, 200)
contours, hierarchy = cv2.findContours(
    canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.imshow('canny', canny)

# find contour
position = []
for cnt in contours:
    cv2.drawContours(imgContour, cnt, -1, (255, 0, 0), 1)
    arcLengthh = cv2.arcLength(cnt, True)
    if arcLengthh > 100 and arcLengthh < 800:
        peri = cv2.arcLength(cnt, False)
        vertices = cv2.approxPolyDP(cnt, peri*0.02, False)
        x, y, w, h = cv2.boundingRect(vertices)
        pos = [x, y, w, h]
        position.append(pos)
        cv2.rectangle(imgContour, (x, y), (x+w, y+h), (0, 255, 0), 4)
cv2.imshow('imgContour', imgContour)

# sort position
position.sort()

# crop img
crop_img = []
for p in position:
    x, y, w, h = p
    crop = img_2[y:y+h, x:x+w]
    crop_img.append(crop)

# convert to mnist
n = 1
for cro in crop_img:
    cro = cv2.resize(cro, (480, 480))
    cro_gray = cv2.cvtColor(cro, cv2.COLOR_BGR2GRAY)
    img_3 = 255 - cro_gray
    img_3 = img_3.astype('float32')
    img_3_min = np.amin(img_3)
    img_4 = img_3 - np.amin(img_3)
    img_5 = 255 * img_4 / (np.amax(img_4))
    kernel = np.ones((5, 5), np.uint8)
    img_6 = cv2.dilate(img_5, kernel, iterations=3)
    img_7 = cv2.resize(img_6, (28, 28), 1)
    img_8 = img_6.astype('uint8')
    cv2.imshow('input'+str(n), img_8)

    x_test_image = np.reshape(img_7, (1, 28, 28))

    # convert 2-D 28x28 image to 4-D nx28x28x1  array
    x_Test4D = x_test_image.reshape(
        x_test_image.shape[0], 28, 28, 1).astype('float32')
    # normalize the image numbers to 0~1
    x_Test4D_normalize = (x_Test4D / np.amax(x_test_image))
    # prediction=model.predict(x_Test4D_normalize)
    prediction = model.predict(x_Test4D_normalize)
    #prediction = (model.predict(x_Test4D_normalize) > 0.5).astype("int32")
    # print(prediction)
    # print(prediction.shape)


    for i in range(prediction.shape[1]):
        if prediction[0, i] > 0.5:
            pre_num = i
    # print(pre_num)
    filename = '%s%s%s' % ('./numbers/number_', str(pre_num), '.jpg')
    img_9 = cv2.imread(filename)
    img_10 = cv2.resize(img_9, (480, 480), 1)
    cv2.imshow('inference'+str(n), img_10)
    n+=1

cv2.waitKey(0)
    